# RAG with BingSearch 

This notebook explains how to integrate Bing Search with the Semantic Kernel  to get the latest information from the internet.

To use Bing Search you simply need a Bing Search API key. You can get the API key by creating a [Bing Search resource](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/create-bing-search-service-resource) in Azure. 

To learn more read the following [documentation](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/overview).


Prepare a Semantic Kernel instance first, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.23.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.23.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.23.0-alpha"
#r "nuget: Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.23.0-alpha"

#!import config/Settings.cs
#!import config/Utils.cs

Installed Packages Microsoft.SemanticKernel, 1.23.0 Microsoft.SemanticKernel.Plugins.Core, 1.23.0-alpha Microsoft.SemanticKernel.Plugins.Web, 1.23.0-alpha Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.23.0

Enter your Bing Search Key in BING_KEY using `InteractiveKernel` method as introduced in [`.NET Interactive`](https://github.com/dotnet/interactive/blob/main/docs/kernels-overview.md)

In [4]:
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

string BING_KEY = (await InteractiveKernel.GetPasswordAsync("Please enter your Bing Search Key")).GetClearTextPassword();

## Basic search plugin

The sample below shows how to create a plugin named SearchPlugin from an instance of `BingTextSearch`. Using `CreateWithSearch` creates a new plugin with a single Search function that calls the underlying text search implementation. The SearchPlugin is added to the Kernel which makes it available to be called during prompt rendering. The prompt template includes a call to `{{SearchPlugin.Search $query}}` which will invoke the SearchPlugin to retrieve results related to the current query. The results are then inserted into the rendered prompt before it is sent to the AI model.

In [5]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Data;
using Microsoft.SemanticKernel.Plugins.Web.Bing;
using Kernel = Microsoft.SemanticKernel.Kernel;

// Create a kernel with OpenAI chat completion
var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);
var kernel = builder.Build();

// Create a text search using Bing search
#pragma warning disable SKEXP0050
var textSearch = new BingTextSearch(apiKey: BING_KEY);

// Build a text search plugin with Bing search and add to the kernel
var searchPlugin = textSearch.CreateWithSearch("SearchPlugin");
kernel.Plugins.Add(searchPlugin);

// Invoke prompt and use text search plugin to provide grounding information
var query = "What is the Semantic Kernel?";
var prompt = "{{SearchPlugin.Search $query}}. {{$query}}";
KernelArguments arguments = new() { { "query", query } };
Console.WriteLine(await kernel.InvokePromptAsync(prompt, arguments));

The Semantic Kernel is a framework developed by Microsoft designed to facilitate the integration of AI capabilities, particularly large language models (LLMs), into applications. It aims to enable developers to leverage these AI models for a variety of tasks, such as natural language understanding, generation, and processing, while also providing tools for managing and orchestrating the interactions with AI systems.

Key features of the Semantic Kernel include:

1. **AI Orchestration**: It helps in managing interactions between different AI components and services, allowing for complex workflows that can involve multiple AI models.

2. **Plugins**: The framework supports the integration of custom plugins, allowing developers to extend the functionality and tailor the AI behavior to specific use cases.

3. **Memory Management**: It allows for the retention of context over interactions, enabling more coherent and contextually aware responses from AI models.

4. **Integration**: The Seman

## Search plugin with citations

The sample below repeats the pattern described in the previous section with a few notable changes:

1. `CreateWithGetTextSearchResults` is used to create a `SearchPlugin` which calls the `GetTextSearchResults` method from the underlying text search implementation.
2. The prompt template uses Handlebars syntax. This allows the template to iterate over the search results and render the name, value and link for each result.
3. The prompt includes an instruction to include citations, so the AI model will do the work of adding citations to the response.

In [6]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Data;
using Microsoft.SemanticKernel.Plugins.Web.Bing;
using Microsoft.SemanticKernel.PromptTemplates.Handlebars;
using Kernel = Microsoft.SemanticKernel.Kernel;

// Create a kernel with OpenAI chat completion
var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);
var kernel = builder.Build();

// Create a text search using Bing search
#pragma warning disable SKEXP0050
var textSearch = new BingTextSearch(apiKey: BING_KEY);

// Build a text search plugin with Bing search and add to the kernel
var searchPlugin = textSearch.CreateWithGetTextSearchResults("SearchPlugin");
kernel.Plugins.Add(searchPlugin);

// Invoke prompt and use text search plugin to provide grounding information
var query = "What is the Semantic Kernel?";
string promptTemplate = """
{{#with (SearchPlugin-GetTextSearchResults query)}}  
    {{#each this}}  
    Name: {{Name}}
    Value: {{Value}}
    Link: {{Link}}
    -----------------
    {{/each}}  
{{/with}}  

{{query}}

Include citations to the relevant information where it is referenced in the response.
""";
KernelArguments arguments = new() { { "query", query } };
HandlebarsPromptTemplateFactory promptTemplateFactory = new();
Console.WriteLine(await kernel.InvokePromptAsync(
    promptTemplate,
    arguments,
    templateFormat: HandlebarsPromptTemplateFactory.HandlebarsTemplateFormat,
    promptTemplateFactory: promptTemplateFactory
));

The Semantic Kernel is a middleware framework developed by Microsoft that integrates Large Language Models (LLMs) with existing APIs and functions. It operates by translating requests made by AI models into function calls within your codebase, allowing the model to leverage existing code actions and return results back to the model for final output generation. Essentially, it facilitates the interaction between AI models and software functions, enabling them to work together seamlessly.

When a request is made to the Semantic Kernel, it marshals that request to the appropriate function and manages the interactions required for the LLM to generate a coherent response based on the outputs from the function calls (Microsoft Learn, "Introduction to Semantic Kernel"). The framework also includes a setup process to establish execution settings that enable automatic function calling, ensuring that the model effectively interacts with the functions in the codebase (Microsoft Learn, "How to qui

## Search plugin with a filter

The samples shown so far will use the top ranked web search results to provide the grounding data. To provide more reliability in the data the web search can be restricted to only return results from a specified site.

The sample below builds on the previous one to add filtering of the search results.
A `TextSearchFilter` with an equality clause is used to specify that only results from the Microsoft Developer Blogs site (`site == 'devblogs.microsoft.com'`) are to be included in the search results.

The sample uses `KernelPluginFactory.CreateFromFunctions` to create the `SearchPlugin`.
A custom description is provided for the plugin.
The `ITextSearch.CreateGetTextSearchResults` extension method is used to create the `KernelFunction` which invokes the text search service.

In [7]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Data;
using Microsoft.SemanticKernel.PromptTemplates.Handlebars;
using Microsoft.SemanticKernel.Plugins.Web.Bing;

// Create a kernel with OpenAI chat completion
var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);
var kernel = builder.Build();

// Create a text search using Bing search
#pragma warning disable SKEXP0050
var textSearch = new BingTextSearch(apiKey: BING_KEY);

// Create a filter to search only the Microsoft Developer Blogs site
#pragma warning disable SKEXP0001
var filter = new TextSearchFilter().Equality("site", "devblogs.microsoft.com");
var searchOptions = new TextSearchOptions() { Filter = filter };

// Build a text search plugin with Bing search and add to the kernel
var searchPlugin = KernelPluginFactory.CreateFromFunctions(
    "SearchPlugin", "Search Microsoft Developer Blogs site only",
    [textSearch.CreateGetTextSearchResults(searchOptions: searchOptions)]);
kernel.Plugins.Add(searchPlugin);

// Invoke prompt and use text search plugin to provide grounding information
var query = "What is the Semantic Kernel?";
string promptTemplate = """
{{#with (SearchPlugin-GetTextSearchResults query)}}  
    {{#each this}}  
    Name: {{Name}}
    Value: {{Value}}
    Link: {{Link}}
    -----------------
    {{/each}}  
{{/with}}  

{{query}}

Include citations to the relevant information where it is referenced in the response.
""";
KernelArguments arguments = new() { { "query", query } };
HandlebarsPromptTemplateFactory promptTemplateFactory = new();
Console.WriteLine(await kernel.InvokePromptAsync(
    promptTemplate,
    arguments,
    templateFormat: HandlebarsPromptTemplateFactory.HandlebarsTemplateFormat,
    promptTemplateFactory: promptTemplateFactory
));


The Semantic Kernel is a production-ready software development kit (SDK) designed to integrate large language models (LLMs) and data stores into applications, thereby enabling the creation of product-scale generative AI (GenAI) solutions. It is an open-source library that aids developers in building AI solutions that can also call existing code, making it highly extensible. The Semantic Kernel supports multiple programming languages, including C#, Python, and Java, which allows for wide applicability across different development environments (Microsoft’s Agentic AI Frameworks: AutoGen and Semantic Kernel, 1).

One of the key features of the Semantic Kernel is its ability to utilize plugins. This functionality allows user requests to be combined with available plugins, enhancing the overall capability of applications built using the SDK. For example, user requests can be processed by agents that communicate with the Semantic Kernel, leveraging the power of plugins and defined personas (